### All Dataset Used before categorised into main_data and additional_data
This is the original url all the dataset are originated in, but you do not need manually clicked this to download as the script below will mitigate that directly
1.  Obesity Data (NCMP) I prepare the dataset using Excel as it is a small dataset
2.  Lookup_table = https://github.com/nooriza16/obesity_prevalence/raw/refs/heads/main/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.zip  
source : https://geoportal.statistics.gov.uk/datasets/9c5ebee4163d435aa4defdaf348ba3c2/about  
3. Mental Health indicator = https://pldr.org/download/2noyv/q3n/samhi_21_01_v5.00_2011_2022_LSOA.csv
4. Data from ONS (all from NOMIS)
5. Data from OHID  
   I download from https://www.localhealth.org.uk/#c=indicator&i=t1.overcrowding&view=map7, however starting of 15/04/2025 the website couldnt be opened. Basically they can provide me with dataset per MSOA 2011 filtered by London directly

In [9]:
# Packagdes for manipulating, processing, and visualizing data
import numpy as np
import scipy.stats as stats
import pandas as pd 
import requests
from zipfile import ZipFile
import os
from urllib.parse import urlparse
import os

# Data from PLDR > SAMHI Index (Aggregating Mental Health LSOA Level to MSOA)

### Look Up Data

In [11]:
# Step 1: Download the file from the URL
url = 'https://github.com/nooriza16/obesity_prevalence/raw/refs/heads/main/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.zip'

# Extract filename from the URL using urllib
filename = os.path.basename(urlparse(url).path)

# Send HTTP request to get the zip file
response = requests.get(url)
response.raise_for_status()  # Check if the request was successful

# Write the content of the response to a local file using the extracted filename
with open(filename, 'wb') as file:
    file.write(response.content)

# Step 2: Extract the zip file
with ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('extracted_files')  # Folder where you want to extract the files

print(f"Zip file '{filename}' downloaded and extracted successfully.")


Zip file 'PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.zip' downloaded and extracted successfully.


In [13]:
import pandas as pd
import os

# Specify the path to the extracted CSV file
csv_file_path = 'extracted_files/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv'

# Read the CSV file into a DataFrame
lookup_table = pd.read_csv(csv_file_path)

lookup_table.drop(columns = ['PCD7', 'PCD8', 'OA11CD', 'LAD11CD', 'LAD11NM', 'LAD11NMW', 'PCDOASPLT'], inplace = True)

C:\Users\nooriza maharani\AppData\Local\Temp\ipykernel_8504\3827118771.py:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lookup_table = pd.read_csv(csv_file_path)


In [15]:
lookup_table.head(2)

,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM
0,E01006686,Liverpool 053E,E02001399,Liverpool 053
1,E01006686,Liverpool 053E,E02001399,Liverpool 053


## SAMHI Index Data by LSOA

In [17]:
lsoa_mhi = pd.read_csv('https://raw.githubusercontent.com/nooriza16/obesity_prevalence/refs/heads/main/samhi_21_01_v5.00_2011_2022_LSOA.csv')

In [19]:
# Select only the desired columns
df_selected = lsoa_mhi[['lsoa11', 'samhi_index.2011']]

In [21]:
df_selected.head(2)

,lsoa11,samhi_index.2011
0,E01000001,-1.757895
1,E01000002,-1.752900


In [23]:
# merge data
merged_data = pd.merge(df_selected, lookup_table, left_on='lsoa11', right_on = 'LSOA11CD')

In [27]:
merged_data.head(2)

,lsoa11,samhi_index.2011,LSOA11CD,LSOA11NM,MSOA11CD,MSOA11NM
0,E01000001,-1.757895,E01000001,City of London 001A,E02000001,City of London 001
1,E01000001,-1.757895,E01000001,City of London 001A,E02000001,City of London 001


In [29]:
merged_data.shape

(1227686, 6)

In [31]:
# aggregate LSOA SAMHI Index into MSOA by grouping the average index
aggregated_data2 = merged_data.groupby('MSOA11CD').agg({'samhi_index.2011': 'mean'}).reset_index()

In [33]:
aggregated_data2.shape

(6791, 2)

In [35]:
##Let's check using our obesity MSOA data which should have 983 rows
main_data = pd.read_csv("https://raw.githubusercontent.com/nooriza16/obesity_prevalence/refs/heads/main/obesity.csv")

In [39]:
main_data.columns

Index(['Code', 'obesity_pre_teen'], dtype='object')

In [41]:
# Merging SAMHI and Obesity data
obesity_samhi = pd.merge(main_data, aggregated_data2, left_on = 'Code', right_on ='MSOA11CD')

In [43]:
obesity_samhi.head(2)

,Code,obesity_pre_teen,MSOA11CD,samhi_index.2011
0,E02000001,0.23214285714285715,E02000001,-1.706400
1,E02000002,0.24015748031496062,E02000002,-1.331989


In [47]:
obesity_samhi.drop(columns = ['MSOA11CD'], inplace = True)

In [49]:
obesity_samhi.shape

(983, 3)

In [51]:
obesity_samhi.to_csv('obesity_samhi.csv', index = 0)

# Data from OHID

In [87]:
OHID_data = pd.read_csv('https://raw.githubusercontent.com/nooriza16/obesity_prevalence/refs/heads/main/Data%20Archive/OHID_Data.csv')

In [142]:
OHID_data.columns

Index(['Code', 'Msoa_name', 'child_poverty', 'overcrowded_houses',
       'self_harm_hospital_admission'],
      dtype='object')

# Data from ONS

In [131]:
ONS = pd.read_csv('https://raw.githubusercontent.com/nooriza16/obesity_prevalence/refs/heads/main/Data%20Archive/ONS_aggregated.csv')

# Merge Additional_data

In [134]:
# merge data
ohid_ons = pd.merge(OHID_data, ONS, on = 'Code', how = 'inner')

In [138]:
# merge data
additional_data = pd.merge(ohid_ons, obesity_samhi, on = 'Code', how = 'inner')

In [144]:
additional_data.shape

(983, 14)

In [148]:
additional_data.drop(columns = ['Msoa_name_y','obesity_pre_teen'], inplace = True)

In [150]:
additional_data.head(2)

,Code,Msoa_name_x,child_poverty,overcrowded_houses,self_harm_hospital_admission,lone_parent_unemployed,families_with_3plus_kids,population_cannot_speak_english,Limiting_long-term_illness_disability,education_no qualifications,work_49hr_more,samhi_index.2011
0,E02000001,City of London 001,7.1,34.6,19.2,24.2,8.816705,1.4,11.5,6.7,1817,-1.706400
1,E02000002,Barking and Dagenham 001,27.2,14.7,62.6,56.0,19.052133,2.4,20.4,32.3,203,-1.331989


In [152]:
additional_data.rename(columns = {'Msoa_name_x' : 'Msoa_name'}, inplace = True)

In [154]:
additional_data.to_csv('additional_data.csv', index = 0)

In [156]:
additional_data.shape

(983, 12)